EE478 Homework 1
----------------

Hansem Ro




Problem 1
---------

1-of-16 decoder:
- inputs: $A[3:0], \bar{A}[3:0]$
- outputs: $word_0, word_1, ..., word_{15}$

Decoder logic:
$
word_0 = \bar{A}[3]\&\bar{A}[2]\&\bar{A}[1]\&\bar{A}[0]\\
word_1= \bar{A}[3]\&\bar{A}[2]\&\bar{A}[1]\&A[0]\\
...\\
word_{15} = A[3]\&A[2]\&A[1]\&A[0]
$

Assumptions:
- 0.2fF/um of wire
- 2 std-cell vertical pitch = 3.6um
- direct decoding
- inverter input cap = 1fF
- output load cap = (4fF * 32bits) = 128fF


In [1]:
# Install numpy and cvxpy
import numpy as np
import cvxpy as cp
from collections import defaultdict


In [2]:
# Table of logical effort for logical units from page 156 of W&H:

def g_inv(n=1):
    return 1

def g_nand(n=2):
    return (n+2)/3

def g_nor(n=2):
    return (2*n+1)/3

# tristate, multiplexer
def g_mux(n=2):
    return 2

# XOR, XNOR
def g_xor(n=2):
    l_half = 2*n*np.ones(n//2)
    range_max = n//2
    if (n % 2 != 0):
        l_half = np.append(l_half,2*n)
        range_max += 1
    for i in range(1,range_max):
        l_half[i] += 2*n*i
    r_half = np.flip(l_half[0:n//2])
    return np.concatenate((l_half, r_half))

# Table of parasitic effort for logical units from page 157 of W&H:

def p_inv(n=1):
    return 1

def p_nand(n=2):
    return n

def p_nor(n=2):
    return n

# tristate, multiplexer
def p_mux(n=1):
    return 2*n

# Common expressions

## Stage expressions

### Electrical effort
def _h(Cout,Cin):
    return Cout/Cin

### Branching effort
def _b(Conpath,Coffpath):
    return (Conpath+Coffpath)/Conpath

### Effort
def _f(g,h):
    return g*h

### Drive
def _x(Cin,g):
    return Cin/g

### Delay
def _d(f,p):
    return f+p

def _d(Cout,x,p):
    return (Cout/x)+p

## Path expressions

### Logical effort
def _G(gs: np.array):
    ret = np.product(gs)

### Electrical effort
def _H(Cout,Cin):
    return Cout/Cin

### Branching effort
def _B(bs: np.array):
    return np.product(bs)

### Effort
def _F(G,B,H):
    return G*B*H

### Effort delay
def _D_F(fs: np.array):
    return np.sum(fs)

### Parasitic delay
def _P(ps: np.array):
    return np.sum(ps)

### Delay
def _D(ds: np.array):
    return np.sum(ds)

def _D(D_F, P):
    return D_F + P

In [3]:
# Testing cvxpy: Book example 4.15 (W&H)
# By: Hansem Ro <hansem7@uw.edu>
#
#                 _____
#--------------->|NAND2|
#--[Inv(x=1)]-+->|(x2) |---+
#             |  |_____|   |     _____
#             +->|NOR2 |   +--->|NOR3 |
#--------------->|(x3) |------->|(x4) |----+----[Inv(x5)]----+
#                |_____|   +--->|_____|    |                 |
#                          |              ===c4=10          ===c5=12
#--------------------------+               |                 |
#                                          v                 v
#
# Expected solutions:
#  Minimum arrival time = 23.44
#  x2 = 1.62
#  x3 = 1.62
#  x4 = 3.37
#  x5 = 6.35

# Step 1: generate delay expressions for each gate
#
# d = f + p = g*h + p = g*(Cout/Cin) + p = (Cout/x) + p
# Cin = g*x

x2 = cp.Variable(pos=True, name="x2")
x3 = cp.Variable(pos=True, name="x3")
x4 = cp.Variable(pos=True, name="x4")
x5 = cp.Variable(pos=True, name="x5")

c4 = 10
c5 = 12

# INV (x=1): branches to NAND2 and NOR2
c_inv_nand2 = g_nand(2)*x2
c_inv_nor2 = g_nor(2)*x3
d_inv1 = p_inv() + (c_inv_nand2+c_inv_nor2)/g_inv()
print("d_inv1 = ", d_inv1)

# NAND2 (x2): connects to NOR3
c_nor3 = g_nor(3)*x4
d_nand2 = p_nand(2) + (c_nor3)/x2
print("d_nand2 = ", d_nand2)

# NOR2 (x3): connects to NOR3
d_nor2 = p_nor(2) + (c_nor3)/x3
print("d_nor2 = ", d_nor2)

# NOR3 (x4): connects to c4 and Inv (x5)
c_nor3_inv = g_inv()*x5
c_nor3_c4 = c4
d_nor3 = p_nor(3) + (c_nor3_inv + c_nor3_c4)/x4
print("d_nor3 = ", d_nor3)

# INV (x5): connects to c5
c_inv_c5 = c5
d_inv2 = p_inv() + (c_inv_c5)/x5
print("d_inv2 = ", d_inv2)

# Step 2: Express as arrival time
a_inv1 = d_inv1
a_nand2 = a_inv1 + d_nand2
a_nor2 = a_inv1 + d_nor2
a_nor3 = cp.maximum(a_nand2,a_nor2) + d_nor3
a_inv2 = a_nor3 + d_inv2
print("a_inv1 = ", a_inv1)
print("a_nand2 = ", a_nand2)
print("a_nor2 = ", a_nor2)
print("a_nor3 = ", a_nor3)
print("a_inv2 = ", a_inv2)

# Step 3: Minimize latest arrival time (a_inv2)

problem = cp.Problem(cp.Minimize(a_inv2))
print(problem.is_dgp(dpp=True))
print(problem.solve(gp=True))
print("x2 = ", x2.value)
print("x3 = ", x3.value)
print("x4 = ", x4.value)
print("x5 = ", x5.value)



d_inv1 =  1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0
d_nand2 =  2.0 + 2.3333333333333335 @ x4 / x2
d_nor2 =  2.0 + 2.3333333333333335 @ x4 / x3
d_nor3 =  3.0 + (1.0 @ x5 + 10.0) / x4
d_inv2 =  1.0 + 12.0 / x5
a_inv1 =  1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0
a_nand2 =  1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x2
a_nor2 =  1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x3
a_nor3 =  maximum(1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x2, 1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x3) + 3.0 + (1.0 @ x5 + 10.0) / x4
a_inv2 =  maximum(1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x2, 1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x3

In [192]:
# logical unit class:
#   At the most fundamental, an output is driven by 1 or more inputs:
#       output <-- unit <-- inputs
#
#   Inputs come from a source or another unit.
#
#   More complex cases involve multiple outputs:
#                    _______
#       output1 <-- |decoder| <-- input1
#          ...  <-- |       | <-- ...
#       outputM <-- |_______| <-- inputN
#
#   To simplify this, we can break the logical unit into smaller units
#   with only 1 output:
#
#       output1 <-- [logicA] <-- inputs
#
#          ...  <-- [logicB] <-- inputs
#
#       output1 <-- [logicC] <-- inputs
#
#   With this output assumption, we can perceive an output as
#   a node of a tree:
#
#       Example:
#                    ____
#       outputH <-- |NAND| <-- inputA
#                   |    | <-- [INV] <-- inputB
#                   |____| <-+
#                            |   ___
#                            +--|NOR| <-- inputC
#                               |___| <-- inputD
#
#
#   We can further simplify this by treating the output net
#   and its logical unit driver as a single node.
#   The inputs will be driven logical unit of type None so
#   that the inputs can be identified as such.
#
class logical_unit:
    def __init__(self, inputs: np.ndarray, output, type=None, Cin=None, drive=None, name=None):
        self.output = output
        self.inputs = inputs
        self.name = name
        self.type = type
        self.type_detailed = None
        self.g = None
        self.p = None
        if isinstance(drive,str):
            self.drive = cp.Variable(pos=True, name=drive)
        else:
            self.drive = drive
        self.Cin = Cin
        if (type is not None and type != "top" and type != "pseudo"):
            if (type != "cap"):
                assert Cin is None
                self.type = "atomic"
                self.type_detailed = type
                if (name is None):
                    self.name = self.type_detailed + str(self.inputs.size) +  "_" + output
                if (type == "inv"):
                    self.g = g_inv()
                    self.p = p_inv()
                elif (type == "nand"):
                    self.g = g_nand(inputs.size)
                    self.p = p_nand(inputs.size)
                elif (type == "nor"):
                    self.g = g_nor(inputs.size)
                    self.p = p_nor(inputs.size)
                elif (type == "mux"):
                    self.g = g_mux(inputs.size)
                    self.p = p_mux(inputs.size)
                # Cin = g*drive
                if (self.drive is None):
                    self.drive = cp.Variable(pos=True, name="x_" + self.name)
                self.Cin = self.g * self.drive
            elif (type == "cap"):
                assert Cin is not None
                if (name is None):
                    self.name = self.type + "_" + self.output

    # print_props: print logical unit properties
    def print_props(self):
        print("name: ", self.name)
        print("type: ", self.type)
        if (self.type == "atomic"):
            print("type_detailed: ", self.type_detailed)
            print("g: ", self.g)
            print("p: ", self.p)
        print("inputs: ", self.inputs)
        print("output: ", self.output)
        print("num_inputs: ", self.inputs.size)
        print("Cin: ", self.Cin)
        print("drive: ", self.drive)

# Class for top level module with known inputs
class top_module(logical_unit):
    def __init__(self, inputs):
        super().__init__(inputs, None, "top")
        print("Creating top level module")
        # create array of output nets
        self.outputs = np.array([])
        # create set for nets
        self.nets = set()
        # create graph for subnodes
        # key: output net
        # value: net driver unit
        self.nodes = defaultdict(list)
        # add inputs to nets set and add pseudo units for inputs to nodes graph
        for input in inputs:
            assert input not in self.nets
            self.nets.add(input)
            tmp = logical_unit(np.array([]), input, "pseudo")
            self.nodes[input].append(tmp)
    
    # add_unit: Add a unit with specified and existing inputs to a new output net
    # @param inputs: str numpy array of input net names 
    # @param output: name of output net
    # @param type: type of logic unit
    def add_unit(self, inputs, output: str, type, Cin=None, drive=None, name=None):
        # Check if all inputs exist
        for input in inputs:
            assert input in self.nets
        # Check if output is a new net
        assert output not in self.nets
        self.nets.add(output)
        tmp = logical_unit(inputs, output, type, Cin=Cin, drive=drive, name=name)
        self.nodes[output].append(tmp)
    
    def add_inv(self, input: str, output: str, drive=None, name=None):
        self.add_unit(np.array([input]), output, "inv", name=name)
    
    def add_cap(self, input: str, output: str, Cin, name=None):
        self.add_unit(np.array([input]), output, "cap", Cin=Cin, name=name)
    
    # get_net: returns net associated to unit name
    # if there is no match, it will return None
    def get_net(self, name):
        for net in self.nets:
            for unit in self.nodes[net]:
                if name == unit.name:
                    return net
        return None

    # get_unit: returns driver unit associated to net
    # or name
    def get_unit(self, net=None, name=None):
        assert net in self.nets
        if (net is not None):
            for unit in self.nodes[net]:
                return unit
        elif (name is not None):
            net = self.get_net(name)
            if (net is not None):
                return self.get_unit(name=net)
        return None

    # generate a fanout list for given net
    def get_fanout(self, net):
        ret = np.array([])
        # iterate through all units and add to return
        # list if its input contains net
        for net_check in self.nodes:
            unit = self.get_unit(net=net_check)
            if net in str(unit.inputs[:]):
                # print("adding unit %s to list" % unit.name)
                ret = np.append(ret, unit.name)
        return ret

    # print_props: print properties of all nodes
    def print_props(self):
        for net in self.nodes:
            for unit in self.nodes[net]:
                unit.print_props()
                print("fanout: ", self.get_fanout(net))
                print()

    # get_cap: returns cap expression at given net
    def get_cap(self, net):
        assert net in self.nets
        assert self.check_module()
        cap = 0
        fanout = self.get_fanout(net)
        for name in fanout:
            ret_name = self.get_net(name)
            if ret_name is not None:
                unit = self.get_unit(net=ret_name)
                cap += unit.Cin
        return cap
    
    # check_module: returns true if all nodes are connected
    # and false otherwise
    def check_module(self):
        visited = set()
        for net in self.nets:
            found = False
            unit = self.get_unit(net=net)
            if unit.type == "cap":
                found = True
                visited.add(net)    
            for tmp in self.nodes:
                if (found):
                    break
                elif (tmp != net):
                    unit_tmp = self.get_unit(net=tmp)
                    if net in str(unit_tmp.inputs[:]):
                        visited.add(net)
                        found = True
                        break
        for net in self.nets:
            if net not in visited:
                return False
        return True

    # __get_a: get arrival time for specified unit name
    def __get_a(self, name):
        assert self.check_module()
        net = self.get_net(name)
        unit = self.get_unit(name=name)
        if (unit.type == "pseudo"):
            return 0
        
        

    # solve: solve and print optimal sizes
    def solve(self):
        assert self.check_module()
        # generate delay statement for each module
        for net in self.nets:
            unit = self.get_unit(net=net)
            if unit.type is not None and unit.type != "pseudo" and unit.type != "cap":
                unit.d = self.get_cap(net)/unit.drive
                if (unit.p is not None):
                    unit.d += unit.p
                print("d_", unit.name, " = ", unit.d)
        # build arrival time expressions while considering fanout
        for net in self.nets:
            unit = self.get_unit(net=net)
            if unit.type is not None and unit.type != "pseudo" and unit.type != "cap":
                unit.a = unit.d
                    


inputs = np.array(["A", "B"])
top = top_module(inputs)
top.add_unit(np.array(["A", "B"]), "C", "nand")
top.print_props()
print(top.check_module())
top.add_cap("C", "D",10)
print(top.check_module())
print(top.get_cap("C"))
print(top.get_cap("A"))

# Algorithm idea: since an input can fan-out to 1 or multiple outputs,
# we can build a tree starting from one of the output nodes.
# Each output node will have an associated arrival time expression.
# We will have a "super" class for output to find the maximum
# arrival time delay among the various outputs.
# The solver will minimize the super class to determine optimal
# solution involving all output nodes.

Creating top level module
name:  None
type:  pseudo
inputs:  []
output:  A
num_inputs:  0
Cin:  None
drive:  None
fanout:  ['nand2_C']

name:  None
type:  pseudo
inputs:  []
output:  B
num_inputs:  0
Cin:  None
drive:  None
fanout:  ['nand2_C']

name:  nand2_C
type:  atomic
type_detailed:  nand
g:  1.3333333333333333
p:  2
inputs:  ['A' 'B']
output:  C
num_inputs:  2
Cin:  1.3333333333333333 @ x_nand2_C
drive:  x_nand2_C
fanout:  []

False
True
10
1.3333333333333333 @ x_nand2_C


In [193]:
# Testing cvxpy: Book example 4.15 (W&H)
# By: Hansem Ro <hansem7@uw.edu>
#
#                   _____
# A--------------->|NAND2|
# B--[Inv(x=1)]-+->|(x2) |---+
#               |  |_____|   |     _____
#               +->|NOR2 |   +--->|NOR3 |
# C--------------->|(x3) |------->|(x4) |----+----[Inv(x5)]----+
#                  |_____|   +--->|_____|    |                 |
#                            |              ===c4=10          ===c5=12
# D--------------------------+               |                 |
#                                            v                 v
#
# Expected solutions:
#  Minimum arrival time = 23.44
#  x2 = 1.62
#  x3 = 1.62
#  x4 = 3.37
#  x5 = 6.35

# Using logical_unit class
inputs = np.array(["A", "B", "C", "D"])
top = top_module(inputs)
top.add_inv("B", "net_inv1", drive=1, name="inv1")
top.add_unit(np.array(["A", "net_inv1"]), "net_nand2", type="nand", drive="x2", name="nand2")
top.add_unit(np.array(["net_inv1", "C"]), "net_nor2", type="nor", drive="x3", name="nor2")
top.add_unit(np.array(["net_nand2", "net_nor2", "D"]), "net_nor3", type="nor", drive="x4")
top.add_cap("net_nor3", "net_c4", 10, name="c4")
top.add_inv("net_nor3", "net_inv2", drive="x5", name="inv2")
top.add_cap("net_inv2", "net_c5", 12, name="c5")
# top.print_props()
# print(top.get_cap("net_inv1"))
top.solve()

Creating top level module
d_ inv2  =  12.0 / x_inv2 + 1.0
d_ inv1  =  (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / x_inv1 + 1.0
d_ nand2  =  2.3333333333333335 @ x4 / x2 + 2.0
d_ nor3_net_nor3  =  (10.0 + 1.0 @ x_inv2) / x4 + 3.0
d_ nor2  =  2.3333333333333335 @ x4 / x3 + 2.0


In [54]:
# Python3 program to print DFS traversal
# from a given given graph

 
# This class represents a directed graph using
# adjacency list representation
class stage:
    def __init__(self, num_inputs, type: str):
        self.type = type
        if (type == "inv"):
            self.g = g_inv()
            self.p = p_inv()
        elif (type == "nand"):
            self.g = g_nand(num_inputs)
            self.p = p_nand(num_inputs)
        elif (type == "nor"):
            self.g = g_nor(num_inputs)
            self.p = p_nor(num_inputs)
        elif (type == "mux"):
            self.g = g_mux(num_inputs)
            self.p = p_mux(num_inputs)
        self.Cin = self.g*x
 
class Graph:
 
    # Constructor
    def __init__(self):
 
        # default dictionary to store graph
        self.graph = defaultdict(list)
 
    # function to add an edge to graph
    def addEdge(self, u, v):
        self.graph[u].append(v)
 
    # A function used by DFS
    def DFSUtil(self, v, visited):
 
        # Mark the current node as visited
        # and print it
        visited.add(v)
        print(v, end=' ')
 
        # Recur for all the vertices
        # adjacent to this vertex
        for neighbour in self.graph[v]:
            if neighbour not in visited:
                self.DFSUtil(neighbour, visited)
 
    # The function to do DFS traversal. It uses
    # recursive DFSUtil()
    def DFS(self, v):
 
        # Create a set to store visited vertices
        visited = set()
 
        # Call the recursive helper function
        # to print DFS traversal
        self.DFSUtil(v, visited)
 
# Driver code
 
 
# Create a graph given
# in the above diagram
g = Graph()
g.addEdge(0, 1)
g.addEdge(0, 2)
g.addEdge(1, 2)
g.addEdge(2, 0)
g.addEdge(1, 3)
g.addEdge(3, 3)
 
print("Following is DFS from (starting from vertex 2)")
g.DFS(2)
 
# This code is contributed by Neelam Yadav

Following is DFS from (starting from vertex 2)
2 0 1 3 

In [137]:


class stage:
    def __init__(self, input: np.ndarray, x: cp.Variable, type: str):
        self.input = input
        self.output = output
        self.type = type
        if (type == "inv"):
            self.g = g_inv()
            self.p = p_inv()
        elif (type == "nand"):
            self.g = g_nand(input.size())
            self.p = p_nand(input.size())
        elif (type == "nor"):
            self.g = g_nor(input.size())
            self.p = p_nor(input.size())
        elif (type == "mux"):
            self.g = g_mux(input.size())
            self.p = p_mux(input.size())
        self.Cin = self.g*x

# inv1 = stage()

# type("hi")

str

In [37]:
class logic:
    # constructor
    def __init__(self, input: np.array, output: np.array, type: str):
        self.input = input
        self.output = output
        self.type = type
        if (type == "inv"):
            self.g = g_inv()
            self.p = p_inv()
        elif (type == "nand"):
            self.g = g_nand(input.size())
            self.p = p_nand(input.size())
        elif (type == "nor"):
            self.g = g_nor(input.size())
            self.p = p_nor(input.size())
        elif (type == "mux"):
            self.g = g_mux(input.size())
            self.p = p_mux(input.size())

class Word:
    decode_logic = 0b0000
    input_cap = cp.Variable() # ? fF

    def set_decode(self, pattern):
        if (pattern >= 0 and pattern < 16):
            self.decode_logic = pattern
        else:
            print("Error")
            self.decode_logic = 0

    
test = Word()
test.set_decode(15)